# Sparse Document Retriever - Experimento

Preencha aqui com detalhes sobre a tarefa.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [9]:
# dataset = "/tmp/data/paracrawl_en_pt_test.csv" #@param {type:"string"}
dataset = "reports_contexts.csv" 
column = "context" #@param {type:"string",label:"Coluna do dataframe de entrada com os contextos para ranquear"}
question = "Qual é o melhor herbicida para erva da ninha ?" #@param {type:"string",label:"Pergunta para a qual os contextos devem ser ranqueados"}

#Hyperparams
retriever_type = "tdidf" #@param ["bm25","tfidf","word2vec"] {type:"string",label:"Tipo de retriever esparço",description:"O retriever pode ser BM25 Okapi, TF-IDF e Word2Vec"}
bm25_k1 = 2 #@param {type:"integer",label:"Argumento k1 do BM25",description:"O argumento k1 do bm25 representa as características da frequência de saturação dos tokens. Ou seja, ele limita o quando cada token pode afetar o score do documento"}
bm25_b = 0.75 #@param {type:"integer",label:"Argumento b do BM25",label:"O argumento b do bm25 é determina o efeito do tamanho de um documento comparado com a média. Quanto maior b maior o efeito}
top = 10 #@param {type:"integer",label:"Quantidade de contextos para retornados"}

In [10]:
if bm25_b<0 or bm25_b>1:
    raise ValueError("O valor de bm25_b deve estar entre 0 e 1")
    
if bm25_k1<1:
    raise ValueError("O valor de bm25_k1 deve ser maior ou igual a 1")

# Acesso ao conjunto de dados¶
O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.
O tipo da variável retornada depende do arquivo de origem:

* pandas.DataFrame para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
* Binary IO stream para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [11]:
import pandas as pd

df = pd.read_csv(dataset)

## Funções de apoio

In [12]:
def build_result_dataframe(sim_contexts_ids,scores,contexts):
    sim_contexts = [contexts[i] for i in sim_contexts_ids[0]]
    df = pd.DataFrame({'doc_id':sim_contexts_ids[0],'score':scores[0],'sim_contexts':sim_contexts})
    df = df.sort_values(by=['score'], ascending=False).reset_index(drop=True)
    return df

## Conteúdo da tarefa

In [13]:
from model_sparse_retriever import TfidfRetriever, W2VRetriever #, BM25Retriever

In [14]:
if retriever_type == "bm25":
    kwargs={'k1':bm25_k1,'b':bm25_b}
    retriever = BM25Retriever(**kwargs)
elif retriever_type == "tfidf":
    retriever = TfidfRetriever()
elif retriever_type == "word2vec":
    from gensim.models import KeyedVectors
    model = KeyedVectors.load_word2vec_format('glove_s300_portugues.txt')
    retriever = W2VRetriever(w2v_model=model)

report_contexts = df[column].to_numpy()
retriever.fit(contexts=report_contexts)
sim_contexts_ids, scores = retriever(questions=question, top=top)

In [15]:
df = build_result_dataframe(sim_contexts_ids,scores,report_contexts)
df.head(10)

,doc_id,score,sim_contexts
0,162,18.098672,"1.3 Palavras-chave: triketonas, sulfoniluréia..."
1,115,18.098672,"1.3 Palavras-chave: triketonas, sulfoniluréia..."
2,689,18.098672,"1.3 Palavras-chave: triketonas, sulfoniluréia..."
3,4,17.896873,"1.3 Palavras-Chave: clomazone, sulfentrazone,..."
4,39,17.896873,"1.3 Palavras-Chave: clomazone, sulfentrazone,..."
5,309,17.819839,"1.3 Palavras-Chave: clodinafop, iodosulfuron,..."
6,840,17.712105,"1.3 Palavras-Chave: trinexapac-ethyl, dose, a..."
7,937,17.663035,"1.3 Palavras-Chave: plante/aplique, sulfentra..."
8,811,17.603450,"1.3 Palavras-Chave: fitotoxicidade, nicosulfu..."
9,1285,17.543288,"1.3 Palavras-Chave: nicosulfuron, fitotoxicid..."


## Salva resultados da tarefa

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.<br>
Use essa pasta para salvar modelos, metadados e outros resultados.

In [16]:
from joblib import dump

artifacts = {
    "model":retriever,
    "report_contexts":report_contexts,
    "question":question,
}

dump(artifacts, "/tmp/data/sparse_retriever.joblib")

['/tmp/data/sparse_retriever.joblib']